In [1]:
import os
import argparse
import gym
import numpy as np
import torch
import torch.cuda
import torch.multiprocessing as _mp

from utils.font_color import color
from models.actor_critic import ActorCritic
from common.atari_wrapper import create_mario_env
from optimizer.sharedadam import SharedAdam
from trainer.a3c.train import train, test
from common.mario_actions import ACTIONS


In [4]:
env_name='SuperMarioBrosNoFrameskip-1-1-v0'
save_path='.//save/trained-models/1-1-basic-reward.pkl'

In [5]:
env = create_mario_env(env_name)
shared_model = ActorCritic(
        env.observation_space.shape[0], len(ACTIONS))

shared_model.share_memory()
shared_model.load_state_dict(torch.load(save_path))


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.uint8'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.uint8'>. Please provide explicit dtype.


In [4]:
from gym.wrappers import Monitor

env = Monitor(env, './video/')

import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

FloatTensor = torch.FloatTensor

cx = Variable(torch.zeros(1, 512),  requires_grad=True ).type(FloatTensor)
with torch.no_grad(): cx=cx
hx = Variable(torch.zeros(1, 512),  requires_grad=True).type(FloatTensor)
with torch.no_grad(): hx=hx

In [5]:
state = env.reset()
state = torch.from_numpy(state)
with torch.no_grad(): state_inp = Variable(state.unsqueeze(0)).type(FloatTensor)
    
for step in range(1000):
    value,action, (hx,cx) = shared_model.forward((state_inp,(hx,cx))) 
    action = np.argmax(action.data.numpy())
    state, reward, done, info = env.step(action)
    state = torch.from_numpy(state)
    with torch.no_grad():
        state_inp = Variable(state.unsqueeze(0)).type(FloatTensor)

    env.render()
    if done : break

env.close()

WARN: <class 'common.atari_wrapper.BufferSkipFrames'> doesn't implement 'reset' method, but it implements deprecated '_reset' method.
